In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import pickle 
import os
import folium
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import sklearn as skl
from sklearn.cluster import KMeans


In [3]:
if not os.path.exists('secret_foursquare_credentials.pkl'):
    Foursquare={}
    Foursquare['ClientID'] = 'TKM0PEJ0HBO0P3PUQC5WHAVLKBLYHBKL1X4D1XQBJTI5P04C'
    Foursquare['ClientSecret'] = 'ONSLHRN2P1E2EWV2QWXVNPNATEY0Y2XWHU2NU2IJWMLA2H0Z'
    with open('secret_foursquare_credentials.pkl','wb') as f:
        pickle.dump(Foursquare, f)
else:
    Foursquare=pickle.load(open('secret_foursquare_credentials.pkl','rb'))

In [4]:
CLIENT_ID = 'TKM0PEJ0HBO0P3PUQC5WHAVLKBLYHBKL1X4D1XQBJTI5P04C'      
CLIENT_SECRET = 'ONSLHRN2P1E2EWV2QWXVNPNATEY0Y2XWHU2NU2IJWMLA2H0Z'   
VERSION = '20180901' # Foursquare API version

In [7]:
with open('/Users/prateek/Desktop/IBM/nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] # define the dataframe columns
neighborhoods = pd.DataFrame(columns=column_names)# create the dataframe
for i in range(len(neighborhoods_data)): #Iterate through each entry in the JSON file
    borough = neighborhoods_data[i]['properties']['borough'] #Boroughs first
    neighborhood_name = neighborhoods_data[i]['properties']['name'] #Followed by Neighborhood name
    neighborhood_lat = neighborhoods_data[i]['properties']['bbox'][1] #Now latitude
    neighborhood_lon = neighborhoods_data[i]['properties']['bbox'][2] #And longitude
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [8]:
mask = neighborhoods['Borough'] == 'Brooklyn' #Make a boolean mask for Manhattan Borough
df = neighborhoods.copy() #Make a copy to leave our original DF intact
df = df[mask] #Now use the mask to eliminate entries not in Manhattan
df.reset_index(inplace = True) #Now reset the index so we can use it
df.drop('index', axis = 1, inplace = True)
df.head() #And take a look at our work so far

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [9]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID = 'TKM0PEJ0HBO0P3PUQC5WHAVLKBLYHBKL1X4D1XQBJTI5P04C' , 
            CLIENT_SECRET = 'ONSLHRN2P1E2EWV2QWXVNPNATEY0Y2XWHU2NU2IJWMLA2H0Z', 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
results = requests.get(url).json()
results['response']['groups'][0]['items'][0]['venue']['id']

NameError: name 'lat' is not defined

In [165]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue ID'          
                            ]
    
    return(nearby_venues)

In [166]:
radius = 500
LIMIT = 20

brooklyn_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [167]:
brooklyn_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa,4b895827f964a5206c2d32e3
1,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar,538ca7d5498ec684c6387a46
2,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop,4ad09cf7f964a520bed820e3
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot,4abe4e84f964a520958c20e3
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place,4ea17d24e5facb29c6d81d6f
5,Bay Ridge,40.625801,-74.030621,XIN,40.625082,-74.030494,Chinese Restaurant,5553702c498ecb85f99d8f37
6,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.623348,-74.031082,Pizza Place,4a343f1ef964a520f69b1fe3
7,Bay Ridge,40.625801,-74.030621,Brooklyn Market,40.626939,-74.029948,Grocery Store,5127e5c7e4b0447b9f67b41d
8,Bay Ridge,40.625801,-74.030621,Karam,40.622931,-74.028316,Middle Eastern Restaurant,4b6785d6f964a520aa532be3
9,Bay Ridge,40.625801,-74.030621,A.L.C. Italian Grocery,40.623051,-74.031224,Grocery Store,5089d4f7e4b0b2a614e157c5


In [168]:
len(brooklyn_venues)

1224

In [458]:
uniques = brooklyn_venues['Venue Category'].unique()

In [459]:
uniques

array(['Caribbean Restaurant', 'Discount Store',
       'Paper / Office Supplies Store', 'Electronics Store', 'Park',
       'Pharmacy', 'Fast Food Restaurant', 'Sandwich Place',
       'Fried Chicken Joint', 'Video Store', 'Bar', 'Lounge', 'Nightclub',
       'Athletics & Sports', 'Deli / Bodega', 'Pool', 'Food',
       'Other Repair Shop', 'Lake', 'Beach', 'Ice Cream Shop',
       'Playground', 'Café', 'Bus Stop', 'Pizza Place',
       'Baseball Stadium', 'Skating Rink', 'Brewery',
       'Monument / Landmark', 'Music Venue',
       'Theme Park Ride / Attraction', 'Other Great Outdoors',
       'Food Court', 'Dessert Shop', 'Clothing Store', 'Gourmet Shop',
       'Surf Spot', 'German Restaurant', 'Burger Joint',
       'Sushi Restaurant', 'Italian Restaurant', 'Hookah Bar',
       'Coffee Shop', 'Bakery', 'Turkish Restaurant', 'Restaurant',
       'Rental Car Location', 'Bubble Tea Shop', 'Cantonese Restaurant',
       'Mobile Phone Shop', 'Peruvian Restaurant', 'Shoe Store',
      

In [460]:
import re
uniqueList = uniques.tolist()
len(uniqueList)

156

In [461]:
targets = ['Restaurant', 'Joint', 'Place', 'Theater', 'Music', 'Club', 'club', 'Cocktail', 'cocktail', 'Roof', \
           'Art', 'Gallery', 'Café', 'Lounge', 'Pub', 'pub', 'Metro', 'Speakeasy', 'House'\
          'Hall', 'Office', 'Coworking', 'Cafe', 'Market', 'Arcade', 'Barcade', 'Auditorium'\
          'Nightlife', 'Party', 'Spa', 'Museum', 'Orchestra', 'Dance']

matching = [s for s in uniqueList if any(xs in s for xs in targets)]
len(matching)

58

In [176]:
mask = brooklyn_venues['Venue Category'].isin(matching)
filtered = brooklyn_venues[mask]
filtered.reset_index(inplace = True)
filtered.drop('index', axis = 1, inplace = True)
filtered.head()

/Users/chrismay/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa,4b895827f964a5206c2d32e3
1,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place,4ea17d24e5facb29c6d81d6f
2,Bay Ridge,40.625801,-74.030621,XIN,40.625082,-74.030494,Chinese Restaurant,5553702c498ecb85f99d8f37
3,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.623348,-74.031082,Pizza Place,4a343f1ef964a520f69b1fe3
4,Bay Ridge,40.625801,-74.030621,Karam,40.622931,-74.028316,Middle Eastern Restaurant,4b6785d6f964a520aa532be3


In [315]:
restaurants = ['Restaurant', 'Joint', 'Place']
matching = [s for s in uniqueList if any(xs in s for xs in restaurants)]
mealMask = filtered['Venue Category'].isin(matching)
nightMask = ~mealMask
eats = filtered[mealMask]
nights = filtered[nightMask]
nights.reset_index(inplace = True)
nights.head()

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa,4b895827f964a5206c2d32e3
1,5,Bay Ridge,40.625801,-74.030621,Mimi Nails,40.622571,-74.031477,Spa,4bd34524462cb7133601de07
2,31,Greenpoint,40.730201,-73.954241,Early,40.732069,-73.954721,Café,54d43863498e653d2ab8343f
3,34,Greenpoint,40.730201,-73.954241,Freckle,40.729991,-73.957129,Spa,4ee8b464754ab0f2605823e9
4,36,Greenpoint,40.730201,-73.954241,Brooklyn Craft Company,40.730357,-73.953139,Arts & Crafts Store,57f599cb498ef66f0428b339


In [ ]:
name = nights[0]
VENUE
url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
    VENUE_ID,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
        )

In [221]:
def getDetails(name, VENUE_ID):
    
    nightVenues=[]
    for name, VENUE_ID in zip(name, VENUE_ID):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
                )
            
        # make the GET request
        results = requests.get(url).json()
        
        # return only relevant information for each nearby venue
        nightVenues.append(results)
        
    return(nightVenues)

    #for i in range(len(details))
    #popularTimes = pd.DataFrame([item for popularHours in popularHours for item in popularHours])
    #popularTimes.columns = ['timeframes',        
                            
night_venues = getDetails(name=nights['Venue'],
                                   VENUE_ID=nights['Venue ID'],
                                  )

Pilo Arts Day Spa and Salon
Mimi Nails
Early
Freckle
Brooklyn Craft Company
Cafe Riviera
Ramona
Living Room Cafe
Lounge Bar Cave
Grace Nails
PASSION CAFE/LOUNGE
Cafe Aziza 7
Jazz. Club
Nina's Nail Spa
F1 Lounge
Jewish Children's Museum
Brooklyn Children's Museum
Chocolatte Crown Heights
Bakerie
Soho Cafe & Grill
Argyle Yarn Shop
Brunswick
Bearded Lady
Madre Mezcaleria
Konditori
Ode to Babel
Cafe Eloise
SummerStage - Betsy Head Park
Silver Factory
Weylin
Bed-Vyne Cocktail
Brown Butter
Brooklyn Historical Society
Henry Public
The Long Island Bar
Anor Spa
Elsa
Fish Tales
Brooklyn Social
Carroll Gardens Greenmarket
Smith Canteen
The Brooklyn Strategist
The Painted Pot
Relaxing On Smith
Brooklyn Botanica
Pioneer Works
Rita
The Bell House
King Killer Studios
Battalion Studios
Brooklyn Art Space
Lucey's Lounge
Fort Greene Park Greenmarket
Dick & Janes
Elan Salon & Day Spa
MTA Subway - Norwood Ave (J/Z)
Kareemation Music Studio
MTA Subway - Shepherd Ave (A/C)
Staples
Matrix Lounge
Podium Night

In [242]:
nightInfo = pd.DataFrame()
for i in range(len(night_venues)):
    nightInfo.loc[i, 'Venue ID'] = night_venues[i]['response']['venue']['id']
    nightInfo.loc[i, 'Likes'] = night_venues[i]['response']['venue']['likes']['count']
    nightInfo.loc[i, 'Tips'] = night_venues[i]['response']['venue']['tips']['count']
    #nightInfo.loc[i, 'Popular Hours'] = night_venues[i]['response']['venue']['popular']['timeframes']

nightInfo.head()

,Venue ID,Likes,Tips
0,4b895827f964a5206c2d32e3,9.0,20.0
1,4bd34524462cb7133601de07,11.0,4.0
2,54d43863498e653d2ab8343f,119.0,27.0
3,4ee8b464754ab0f2605823e9,26.0,13.0
4,57f599cb498ef66f0428b339,7.0,3.0


In [245]:
def getHours(name, VENUE_ID):
    
    nightHours=[]
    for name, VENUE_ID in zip(name, VENUE_ID):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}/hours??&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
                )
            
        # make the GET request
        nightResults = requests.get(url).json()
        
        # return only relevant information for each nearby venue
        nightHours.append(nightResults)
        
    return(nightHours)

    #for i in range(len(details))
    #popularTimes = pd.DataFrame([item for popularHours in popularHours for item in popularHours])
    #popularTimes.columns = ['timeframes',        
                            
night_hours = getHours(name=nights['Venue'],
                                   VENUE_ID=nights['Venue ID'],
                                  )

Pilo Arts Day Spa and Salon
Mimi Nails
Early
Freckle
Brooklyn Craft Company
Cafe Riviera
Ramona
Living Room Cafe
Lounge Bar Cave
Grace Nails
PASSION CAFE/LOUNGE
Cafe Aziza 7
Jazz. Club
Nina's Nail Spa
F1 Lounge
Jewish Children's Museum
Brooklyn Children's Museum
Chocolatte Crown Heights
Bakerie
Soho Cafe & Grill
Argyle Yarn Shop
Brunswick
Bearded Lady
Madre Mezcaleria
Konditori
Ode to Babel
Cafe Eloise
SummerStage - Betsy Head Park
Silver Factory
Weylin
Bed-Vyne Cocktail
Brown Butter
Brooklyn Historical Society
Henry Public
The Long Island Bar
Anor Spa
Elsa
Fish Tales
Brooklyn Social
Carroll Gardens Greenmarket
Smith Canteen
The Brooklyn Strategist
The Painted Pot
Relaxing On Smith
Brooklyn Botanica
Pioneer Works
Rita
The Bell House
King Killer Studios
Battalion Studios
Brooklyn Art Space
Lucey's Lounge
Fort Greene Park Greenmarket
Dick & Janes
Elan Salon & Day Spa
MTA Subway - Norwood Ave (J/Z)
Kareemation Music Studio
MTA Subway - Shepherd Ave (A/C)
Staples
Matrix Lounge
Podium Night

In [284]:
for i in range(len(night_hours)):
    print(i)
    try:
        nightInfo.loc[i, 'Hours'] = night_hours[i]['response']['hours']
    except:
        nightInfo.loc[i, 'Hours'] = 'N/A'
    
    try:
        nightInfo.loc[i, 'Popular'] = night_hours[i]['response']['popular']['timeframes']
    except:
        nightInfo.loc[i, 'Popular'] = 'N/A'

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


In [308]:
nightInfo.drop('Popular', axis = 1, inplace = True)

In [313]:
type(nights['Venue ID'][0])

str

In [332]:
info = nights.join(nightInfo, lsuffix='l', rsuffix='r', how='inner', sort=False)

In [333]:
info.head()

,Neighborhood,Venue,Venue Category,Venue IDl,Venue IDr,Likes,Tips
0,Bay Ridge,Pilo Arts Day Spa and Salon,Spa,4b895827f964a5206c2d32e3,4b895827f964a5206c2d32e3,9.0,20.0
1,Bay Ridge,Mimi Nails,Spa,4bd34524462cb7133601de07,4bd34524462cb7133601de07,11.0,4.0
2,Greenpoint,Early,Café,54d43863498e653d2ab8343f,54d43863498e653d2ab8343f,119.0,27.0
3,Greenpoint,Freckle,Spa,4ee8b464754ab0f2605823e9,4ee8b464754ab0f2605823e9,26.0,13.0
4,Greenpoint,Brooklyn Craft Company,Arts & Crafts Store,57f599cb498ef66f0428b339,57f599cb498ef66f0428b339,7.0,3.0


In [336]:
info.head()

,Neighborhood,Venue,Venue Category,Venue IDl,Venue IDr,Likes,Tips
0,Bay Ridge,Pilo Arts Day Spa and Salon,Spa,4b895827f964a5206c2d32e3,4b895827f964a5206c2d32e3,9.0,20.0
1,Bay Ridge,Mimi Nails,Spa,4bd34524462cb7133601de07,4bd34524462cb7133601de07,11.0,4.0
2,Greenpoint,Early,Café,54d43863498e653d2ab8343f,54d43863498e653d2ab8343f,119.0,27.0
3,Greenpoint,Freckle,Spa,4ee8b464754ab0f2605823e9,4ee8b464754ab0f2605823e9,26.0,13.0
4,Greenpoint,Brooklyn Craft Company,Arts & Crafts Store,57f599cb498ef66f0428b339,57f599cb498ef66f0428b339,7.0,3.0


In [341]:
info.describe()

,Likes,Tips
count,114.000000,114.000000
mean,102.192982,23.447368
std,188.765992,35.423629
min,0.000000,0.000000
25%,7.000000,3.000000
50%,22.500000,8.000000
75%,104.500000,24.750000
max,1084.000000,185.000000


In [370]:
popularPlacesMask = info['Likes'] > 22.5
hotTipsMask = info['Tips'] > 8
combined = popularPlacesMask | hotTipsMask
popularPlaces = info[combined]
len(popularPlaces)
popularPlaces.head()

,Neighborhood,Venue,Venue Category,Venue IDl,Venue IDr,Likes,Tips
0,Bay Ridge,Pilo Arts Day Spa and Salon,Spa,4b895827f964a5206c2d32e3,4b895827f964a5206c2d32e3,9.0,20.0
2,Greenpoint,Early,Café,54d43863498e653d2ab8343f,54d43863498e653d2ab8343f,119.0,27.0
3,Greenpoint,Freckle,Spa,4ee8b464754ab0f2605823e9,4ee8b464754ab0f2605823e9,26.0,13.0
5,Greenpoint,Cafe Riviera,Café,4a6b3156f964a52025ce1fe3,4a6b3156f964a52025ce1fe3,28.0,23.0
6,Greenpoint,Ramona,Cocktail Bar,52c4d86911d22261f6be51d8,52c4d86911d22261f6be51d8,554.0,110.0


In [400]:
popularCount = popularPlaces.groupby(['Neighborhood'], as_index = False).count()
popularCount

,Neighborhood,Venue,Venue Category,Venue IDl,Venue IDr,Likes,Tips
0,Bay Ridge,1,1,1,1,1,1
1,Bedford Stuyvesant,1,1,1,1,1,1
2,Boerum Hill,2,2,2,2,2,2
3,Brooklyn Heights,1,1,1,1,1,1
4,Carroll Gardens,5,5,5,5,5,5
5,Clinton Hill,1,1,1,1,1,1
6,Cobble Hill,4,4,4,4,4,4
7,Coney Island,1,1,1,1,1,1
8,Crown Heights,2,2,2,2,2,2
9,Ditmas Park,2,2,2,2,2,2


In [412]:
countMask = popularCount['Venue'] > 1
popNeighborhoods = popularCount[countMask]
neighList = popNeighborhoods['Neighborhood'].tolist()

['Boerum Hill',
 'Carroll Gardens',
 'Cobble Hill',
 'Crown Heights',
 'Ditmas Park',
 'Downtown',
 'East Williamsburg',
 'Fort Greene',
 'Gowanus',
 'Gravesend',
 'Greenpoint',
 'Prospect Heights',
 'Prospect Lefferts Gardens',
 'Red Hook',
 'Vinegar Hill',
 'Weeksville',
 'Windsor Terrace']

Now that we have the list of most popular neighborhoods, let's make a new dataframe that only contains venues from that are located in the popular neighborhoods

In [435]:
filtered.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa,4b895827f964a5206c2d32e3
1,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place,4ea17d24e5facb29c6d81d6f
2,Bay Ridge,40.625801,-74.030621,XIN,40.625082,-74.030494,Chinese Restaurant,5553702c498ecb85f99d8f37
3,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.623348,-74.031082,Pizza Place,4a343f1ef964a520f69b1fe3
4,Bay Ridge,40.625801,-74.030621,Karam,40.622931,-74.028316,Middle Eastern Restaurant,4b6785d6f964a520aa532be3


In [437]:
hoodMask = filtered['Neighborhood'].isin(neighList)
popVenues = filtered[hoodMask]
neighMask = df['Neighborhood'].isin(neighList)
popNeighborhoods = df[neighMask]

In [438]:
popNeighborhoods

,Borough,Neighborhood,Latitude,Longitude
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471
9,Brooklyn,Crown Heights,40.670829,-73.943291
12,Brooklyn,Windsor Terrace,40.656946,-73.980073
13,Brooklyn,Prospect Heights,40.676822,-73.964859
19,Brooklyn,Cobble Hill,40.687920,-73.998561
20,Brooklyn,Carroll Gardens,40.680540,-73.994654
21,Brooklyn,Red Hook,40.676253,-74.012759
22,Brooklyn,Gowanus,40.673931,-73.994441
23,Brooklyn,Fort Greene,40.688527,-73.972906


In [486]:
radius = 500
LIMIT = 100

hot_venues = getNearbyVenues(names=popNeighborhoods['Neighborhood'],
                                   latitudes=popNeighborhoods['Latitude'],
                                   longitudes=popNeighborhoods['Longitude']
                                  )
len(hot_venues)

Greenpoint
Gravesend
Crown Heights
Windsor Terrace
Prospect Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Downtown
Boerum Hill
Prospect Lefferts Gardens
East Williamsburg
Ditmas Park
Vinegar Hill
Weeksville


1042

In [487]:
uniques = hot_venues['Venue Category'].unique()
hotUniqueList = uniques.tolist()
hotUniqueList

['Yoga Studio',
 'Laundry Service',
 'Polish Restaurant',
 'Mexican Restaurant',
 'Bar',
 'Café',
 'Pizza Place',
 'Grocery Store',
 'Gymnastics Gym',
 'Spa',
 'Fried Chicken Joint',
 'Arts & Crafts Store',
 'Record Shop',
 'Deli / Bodega',
 'Italian Restaurant',
 'Cocktail Bar',
 'Furniture / Home Store',
 'Coffee Shop',
 'Bookstore',
 'Beer Store',
 'Gastropub',
 'Restaurant',
 'Liquor Store',
 'Boutique',
 'Sandwich Place',
 'Wine Shop',
 'Sushi Restaurant',
 'Vegetarian / Vegan Restaurant',
 'New American Restaurant',
 'Vietnamese Restaurant',
 'Vape Store',
 'Salon / Barbershop',
 'Whisky Bar',
 'Thai Restaurant',
 'Fruit & Vegetable Store',
 'Tea Room',
 'Ice Cream Shop',
 'Beer Bar',
 'Bakery',
 'Nightclub',
 'French Restaurant',
 'Bagel Shop',
 'Food Truck',
 'Optical Shop',
 'American Restaurant',
 'Coworking Space',
 'Donut Shop',
 'Juice Bar',
 'Ramen Restaurant',
 'Dance Studio',
 'Wine Bar',
 'Antique Shop',
 'Nail Salon',
 'Falafel Restaurant',
 'Playground',
 'Burger Joi

In [488]:
newtargets = ['Restaurant', 'Joint', 'Place', 'Theater', 'Music', 'Club', 'club', 'Cocktail', 'cocktail', 'Roof', \
           'Art', 'Gallery', 'Café', 'Lounge', 'Pub', 'pub', 'Metro', 'Speakeasy', 'House'\
          'Hall', 'Office', 'Coworking', 'Cafe', 'Market', 'Arcade', 'Barcade', 'Auditorium'\
          'Nightlife', 'Party', 'Spa', 'Museum', 'Orchestra', 'Dance', 'Record', 'Boutique', 'Pie', 'Beer', 'Video'\
          'Event', 'Movie', 'Concert', 'Hostel', 'Hotel', 'Laser']

hotMatches = [s for s in hotUniqueList if any(xs in s for xs in newtargets)]
len(hotMatches)

102

In [484]:
mask = hot_venues['Venue Category'].isin(matching)
hotFilter = hot_venues[mask]
hotFilter.reset_index(inplace = True)
hotFilter.drop('index', axis = 1, inplace = True)
hotFilter.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Greenpoint,40.730201,-73.954241,Karczma,40.730102,-73.955092,Polish Restaurant,4a916658f964a520151a20e3
1,Greenpoint,40.730201,-73.954241,Oxomoco,40.729981,-73.955460,Mexican Restaurant,5ae8ed8eb3c961002b3360da
2,Greenpoint,40.730201,-73.954241,Early,40.732069,-73.954721,Café,54d43863498e653d2ab8343f
3,Greenpoint,40.730201,-73.954241,Fornino,40.728951,-73.953862,Pizza Place,519bdb24abd81726b7a90ede
4,Greenpoint,40.730201,-73.954241,Friducha,40.731512,-73.954281,Mexican Restaurant,5a2b5dbdc0af5755f9b41a7a


In [478]:
hotRestaurants = ['Restaurant', 'Joint', 'Place']
hotRestaurantsMatch = [s for s in hotUniqueList if any(xs in s for xs in hotRestaurants)]
hotRestMask = hotFilter['Venue Category'].isin(hotRestaurantsMatch)
hotNightMask = ~hotRestMask
hotEats = hotFilter[hotRestMask]
hotNights = hotFilter[hotNightMask]
hotNights.reset_index(inplace = True)
hotNights.drop('index', axis = 1, inplace = True)
hotNights.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Greenpoint,40.730201,-73.954241,Early,40.732069,-73.954721,Café,54d43863498e653d2ab8343f
1,Greenpoint,40.730201,-73.954241,Freckle,40.729991,-73.957129,Spa,4ee8b464754ab0f2605823e9
2,Greenpoint,40.730201,-73.954241,Cafe Riviera,40.728514,-73.953477,Café,4a6b3156f964a52025ce1fe3
3,Greenpoint,40.730201,-73.954241,Ramona,40.730249,-73.957739,Cocktail Bar,52c4d86911d22261f6be51d8
4,Greenpoint,40.730201,-73.954241,The Brew Inn,40.731031,-73.954191,Gastropub,5285412911d2a3e51484ff56


In [490]:
len(hotNights), len(hotEats)

(87, 180)

In [532]:
hotNightCounts = hotNights.groupby(['Neighborhood'], as_index = False).count()
hotEatCounts = hotEats.groupby(['Neighborhood'], as_index = False).count()
hotEatCounts.sort_values(by = ['Neighborhood'], axis = 0, ascending = True, inplace = True)
hotNightCounts.sort_values(by = ['Neighborhood'], axis = 0, ascending = True, inplace = True)

In [533]:
hotNightCounts

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Boerum Hill,7,7,7,7,7,7,7
1,Carroll Gardens,5,5,5,5,5,5,5
2,Cobble Hill,5,5,5,5,5,5,5
3,Crown Heights,2,2,2,2,2,2,2
4,Ditmas Park,2,2,2,2,2,2,2
5,Downtown,7,7,7,7,7,7,7
6,East Williamsburg,7,7,7,7,7,7,7
7,Fort Greene,6,6,6,6,6,6,6
8,Gowanus,5,5,5,5,5,5,5
9,Gravesend,6,6,6,6,6,6,6


In [515]:
popNeighborhoods.sort_values(by = ['Neighborhood'], axis = 0, ascending = True, inplace = True)


/Users/chrismay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,index,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts
11,41,Brooklyn,Boerum Hill,40.685683,-73.983748,6,15
6,20,Brooklyn,Carroll Gardens,40.680540,-73.994654,7,5
5,19,Brooklyn,Cobble Hill,40.687920,-73.998561,7,15
2,9,Brooklyn,Crown Heights,40.670829,-73.943291,5,17
14,54,Brooklyn,Ditmas Park,40.643675,-73.961013,3,2
10,40,Brooklyn,Downtown,40.690844,-73.983463,8,12
13,49,Brooklyn,East Williamsburg,40.708492,-73.938858,6,9
9,23,Brooklyn,Fort Greene,40.688527,-73.972906,6,7
8,22,Brooklyn,Gowanus,40.673931,-73.994441,5,10
1,4,Brooklyn,Gravesend,40.595260,-73.973471,5,12


In [520]:
popNeighborhoods.reset_index(inplace = True)

In [522]:
popNeighborhoods.drop('index', axis = 1, inplace = True)
popNeighborhoods

,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts
0,Brooklyn,Boerum Hill,40.685683,-73.983748,6,15
1,Brooklyn,Carroll Gardens,40.680540,-73.994654,7,5
2,Brooklyn,Cobble Hill,40.687920,-73.998561,7,15
3,Brooklyn,Crown Heights,40.670829,-73.943291,5,17
4,Brooklyn,Ditmas Park,40.643675,-73.961013,3,2
5,Brooklyn,Downtown,40.690844,-73.983463,8,12
6,Brooklyn,East Williamsburg,40.708492,-73.938858,6,9
7,Brooklyn,Fort Greene,40.688527,-73.972906,6,7
8,Brooklyn,Gowanus,40.673931,-73.994441,5,10
9,Brooklyn,Gravesend,40.595260,-73.973471,5,12


In [534]:
popNeighborhoods['Night Counts'] = hotNightCounts['Venue']
popNeighborhoods['Restaurant Counts'] = hotEatCounts['Venue']
popNeighborhoods

/Users/chrismay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/chrismay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts
0,Brooklyn,Boerum Hill,40.685683,-73.983748,7,13,84
1,Brooklyn,Carroll Gardens,40.680540,-73.994654,5,12,100
2,Brooklyn,Cobble Hill,40.687920,-73.998561,5,17,100
3,Brooklyn,Crown Heights,40.670829,-73.943291,2,4,19
4,Brooklyn,Ditmas Park,40.643675,-73.961013,2,14,50
5,Brooklyn,Downtown,40.690844,-73.983463,7,15,100
6,Brooklyn,East Williamsburg,40.708492,-73.938858,7,5,71
7,Brooklyn,Fort Greene,40.688527,-73.972906,6,20,79
8,Brooklyn,Gowanus,40.673931,-73.994441,5,10,59
9,Brooklyn,Gravesend,40.595260,-73.973471,6,7,30


In [526]:
hotNeighCounts = hot_venues.groupby(['Neighborhood'], as_index = False).count()

In [535]:
popNeighborhoods['Total Counts'] = hotNeighCounts['Venue']
popNeighborhoods

/Users/chrismay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts
0,Brooklyn,Boerum Hill,40.685683,-73.983748,7,13,84
1,Brooklyn,Carroll Gardens,40.680540,-73.994654,5,12,100
2,Brooklyn,Cobble Hill,40.687920,-73.998561,5,17,100
3,Brooklyn,Crown Heights,40.670829,-73.943291,2,4,19
4,Brooklyn,Ditmas Park,40.643675,-73.961013,2,14,50
5,Brooklyn,Downtown,40.690844,-73.983463,7,15,100
6,Brooklyn,East Williamsburg,40.708492,-73.938858,7,5,71
7,Brooklyn,Fort Greene,40.688527,-73.972906,6,20,79
8,Brooklyn,Gowanus,40.673931,-73.994441,5,10,59
9,Brooklyn,Gravesend,40.595260,-73.973471,6,7,30


In [538]:
popNeighborhoods['Night Frequency'] = popNeighborhoods['Night Counts'] / popNeighborhoods['Total Counts']
popNeighborhoods['Eats Frequency'] = popNeighborhoods['Restaurant Counts'] / popNeighborhoods['Total Counts']
popNeighborhoods

/Users/chrismay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/chrismay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts,Night Frequency,Eats Frequency
0,Brooklyn,Boerum Hill,40.685683,-73.983748,7,13,84,0.083333,0.154762
1,Brooklyn,Carroll Gardens,40.680540,-73.994654,5,12,100,0.050000,0.120000
2,Brooklyn,Cobble Hill,40.687920,-73.998561,5,17,100,0.050000,0.170000
3,Brooklyn,Crown Heights,40.670829,-73.943291,2,4,19,0.105263,0.210526
4,Brooklyn,Ditmas Park,40.643675,-73.961013,2,14,50,0.040000,0.280000
5,Brooklyn,Downtown,40.690844,-73.983463,7,15,100,0.070000,0.150000
6,Brooklyn,East Williamsburg,40.708492,-73.938858,7,5,71,0.098592,0.070423
7,Brooklyn,Fort Greene,40.688527,-73.972906,6,20,79,0.075949,0.253165
8,Brooklyn,Gowanus,40.673931,-73.994441,5,10,59,0.084746,0.169492
9,Brooklyn,Gravesend,40.595260,-73.973471,6,7,30,0.200000,0.233333


In [544]:
restaurants = hotEats['Venue Category'].unique()
restList = restaurants.tolist()
restList

['Polish Restaurant',
 'Mexican Restaurant',
 'Pizza Place',
 'Fried Chicken Joint',
 'Italian Restaurant',
 'Restaurant',
 'Sandwich Place',
 'Sushi Restaurant',
 'Thai Restaurant',
 'Chinese Restaurant',
 'Burger Joint',
 'American Restaurant',
 'Middle Eastern Restaurant',
 'French Restaurant',
 'Caribbean Restaurant',
 'Latin American Restaurant',
 'Cajun / Creole Restaurant',
 'Southern / Soul Food Restaurant',
 'Spanish Restaurant',
 'Japanese Restaurant',
 'Seafood Restaurant',
 'Dumpling Restaurant',
 'Cuban Restaurant',
 'BBQ Joint',
 'Indian Restaurant',
 'Tapas Restaurant',
 'German Restaurant',
 'South American Restaurant',
 'Asian Restaurant',
 'Burrito Place',
 'Taco Place']

In [548]:
chineseTargets = ['Chinese', 'Hong Kong', 'Taiwan', 'Noodle', 'Ramen', 'Dumpling', 'Asian', 'Japanese']

chineseMatch = [s for s in restList if any(xs in s for xs in chineseTargets)]

In [549]:
chineseMask = hotEats['Venue Category'].isin(chineseMatch)
chineseFilter = hotEats[chineseMask]
chineseFilter.reset_index(inplace = True)
chineseFilter.drop('index', axis = 1, inplace = True)
chineseFilter.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Gravesend,40.595260,-73.973471,Avenue U Eagle Kitchen,40.596568,-73.978874,Chinese Restaurant,4c03eff80b8eef3b64d3b882
1,Gravesend,40.595260,-73.973471,Choy Le Chinese Restaraunt,40.597537,-73.968531,Chinese Restaurant,4db9e9370437ffd20796f4af
2,Windsor Terrace,40.656946,-73.980073,East Wind Snack Shop,40.660297,-73.980169,Chinese Restaurant,54e39b14498e9c646b42958e
3,Cobble Hill,40.687920,-73.998561,Hibino,40.690076,-73.996497,Japanese Restaurant,48a41073f964a52091511fe3
4,Cobble Hill,40.687920,-73.998561,Oh! Dumplings,40.687722,-73.992943,Dumpling Restaurant,5b7f53abe727c40024124d8c


In [555]:
chiNeighborCounts = chineseFilter.groupby(['Neighborhood'], as_index = False).count()
chiNeighborCounts

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Boerum Hill,2,2,2,2,2,2,2
1,Cobble Hill,2,2,2,2,2,2,2
2,Ditmas Park,3,3,3,3,3,3,3
3,Downtown,5,5,5,5,5,5,5
4,Fort Greene,1,1,1,1,1,1,1
5,Gowanus,1,1,1,1,1,1,1
6,Gravesend,2,2,2,2,2,2,2
7,Prospect Lefferts Gardens,1,1,1,1,1,1,1
8,Weeksville,1,1,1,1,1,1,1
9,Windsor Terrace,1,1,1,1,1,1,1


In [557]:
popNeighborhoods.drop('Chinese Restaurants', axis = 1, inplace = True)

In [563]:
for i in range(len(chiNeighborCounts)):
    place = chiNeighborCounts.loc[i, 'Neighborhood']
    row = popNeighborhoods.loc[popNeighborhoods['Neighborhood']==place].index
    popNeighborhoods.loc[row, 'Chinese Restaurants'] = chiNeighborCounts.loc[i, 'Venue']

popNeighborhoods.fillna(0.0)

,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts,Night Frequency,Eats Frequency,Chinese Restaurants
0,Brooklyn,Boerum Hill,40.685683,-73.983748,7,13,84,0.083333,0.154762,2.0
1,Brooklyn,Carroll Gardens,40.680540,-73.994654,5,12,100,0.050000,0.120000,0.0
2,Brooklyn,Cobble Hill,40.687920,-73.998561,5,17,100,0.050000,0.170000,2.0
3,Brooklyn,Crown Heights,40.670829,-73.943291,2,4,19,0.105263,0.210526,0.0
4,Brooklyn,Ditmas Park,40.643675,-73.961013,2,14,50,0.040000,0.280000,3.0
5,Brooklyn,Downtown,40.690844,-73.983463,7,15,100,0.070000,0.150000,5.0
6,Brooklyn,East Williamsburg,40.708492,-73.938858,7,5,71,0.098592,0.070423,0.0
7,Brooklyn,Fort Greene,40.688527,-73.972906,6,20,79,0.075949,0.253165,1.0
8,Brooklyn,Gowanus,40.673931,-73.994441,5,10,59,0.084746,0.169492,1.0
9,Brooklyn,Gravesend,40.595260,-73.973471,6,7,30,0.200000,0.233333,2.0


In [566]:
popNeighborhoods['Chinese Frequency'] = popNeighborhoods['Chinese Restaurants'] / popNeighborhoods['Total Counts']

/Users/chrismay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [568]:
popNeighborhoods.fillna(0.0)

,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts,Night Frequency,Eats Frequency,Chinese Restaurants,Chinese Frequency
0,Brooklyn,Boerum Hill,40.685683,-73.983748,7,13,84,0.083333,0.154762,2.0,0.023810
1,Brooklyn,Carroll Gardens,40.680540,-73.994654,5,12,100,0.050000,0.120000,0.0,0.000000
2,Brooklyn,Cobble Hill,40.687920,-73.998561,5,17,100,0.050000,0.170000,2.0,0.020000
3,Brooklyn,Crown Heights,40.670829,-73.943291,2,4,19,0.105263,0.210526,0.0,0.000000
4,Brooklyn,Ditmas Park,40.643675,-73.961013,2,14,50,0.040000,0.280000,3.0,0.060000
5,Brooklyn,Downtown,40.690844,-73.983463,7,15,100,0.070000,0.150000,5.0,0.050000
6,Brooklyn,East Williamsburg,40.708492,-73.938858,7,5,71,0.098592,0.070423,0.0,0.000000
7,Brooklyn,Fort Greene,40.688527,-73.972906,6,20,79,0.075949,0.253165,1.0,0.012658
8,Brooklyn,Gowanus,40.673931,-73.994441,5,10,59,0.084746,0.169492,1.0,0.016949
9,Brooklyn,Gravesend,40.595260,-73.973471,6,7,30,0.200000,0.233333,2.0,0.066667


In [609]:
clusterDF = popNeighborhoods.copy()
clusterDF.drop('Borough', axis = 1, inplace = True)
clusterDF.drop('Neighborhood', axis = 1, inplace = True)
clusterDF.drop('Latitude', axis = 1, inplace = True)
clusterDF.drop('Longitude', axis = 1, inplace = True)
clusterDF.drop('Restaurant Counts', axis = 1, inplace = True)
clusterDF.drop('Night Counts', axis = 1, inplace = True)

In [610]:
clusterDF.fillna(0.0, inplace = True)
clusterDF['Chinese Frequency'].astype(float)

0     0.023810
1     0.000000
2     0.020000
3     0.000000
4     0.060000
5     0.050000
6     0.000000
7     0.012658
8     0.016949
9     0.066667
10    0.000000
11    0.000000
12    0.020000
13    0.000000
14    0.000000
15    0.066667
16    0.040000
Name: Chinese Frequency, dtype: float64

In [595]:
type(clusterDF.loc[1, 'Chinese Frequency'])

numpy.float64

In [611]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clusterDF)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 3, 2, 0, 3, 1, 1, 0, 2], dtype=int32)

In [612]:
popNeighborhoods['Cluster'] = kmeans.labels_

/Users/chrismay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [613]:
popNeighborhoods

,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts,Night Frequency,Eats Frequency,Chinese Restaurants,Chinese Frequency,Cluster
0,Brooklyn,Boerum Hill,40.685683,-73.983748,7,13,84,0.083333,0.154762,2.0,0.023810,1
1,Brooklyn,Carroll Gardens,40.680540,-73.994654,5,12,100,0.050000,0.120000,NaN,NaN,3
2,Brooklyn,Cobble Hill,40.687920,-73.998561,5,17,100,0.050000,0.170000,2.0,0.020000,3
3,Brooklyn,Crown Heights,40.670829,-73.943291,2,4,19,0.105263,0.210526,NaN,NaN,2
4,Brooklyn,Ditmas Park,40.643675,-73.961013,2,14,50,0.040000,0.280000,3.0,0.060000,0
5,Brooklyn,Downtown,40.690844,-73.983463,7,15,100,0.070000,0.150000,5.0,0.050000,3
6,Brooklyn,East Williamsburg,40.708492,-73.938858,7,5,71,0.098592,0.070423,NaN,NaN,1
7,Brooklyn,Fort Greene,40.688527,-73.972906,6,20,79,0.075949,0.253165,1.0,0.012658,1
8,Brooklyn,Gowanus,40.673931,-73.994441,5,10,59,0.084746,0.169492,1.0,0.016949,0
9,Brooklyn,Gravesend,40.595260,-73.973471,6,7,30,0.200000,0.233333,2.0,0.066667,2


Cluster 2 seems to have a high number of nightlife venues, with very low numbers of Chinese Restaurants, and a balanced number of other Restaurants. This cluster is a good starting point to look for a new location for our restaurant

In [614]:
# create map
latitude = 40.685683
longitude = -73.983748

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(popNeighborhoods['Latitude'], popNeighborhoods['Longitude'], popNeighborhoods['Neighborhood'], popNeighborhoods['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters